In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='add the key',  # Replace this with your real key
    model_name="llama3-70b-8192"  # Correct model name
)

response = llm.invoke("The first person to land on the moon was ...")
print(response.content)


Neil Armstrong! On July 20, 1969, Neil Armstrong became the first person to set foot on the moon, famously declaring "That's one small step for man, one giant leap for mankind" as he stepped off the lunar module Eagle onto the moon's surface.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://internshala.com/job/detail/fresher-remote-junior-data-scientist-job-at-mediacom-india1753532936")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.



















 




Junior Data Scientist Remote Fresher Job




























 





















Close


















Cancel
Close


















Close
Close


















Close















Cancel
Yes












        Oops, your browser does not support Javascript. Please use Internshala in another browser.
    




            Loading, please wait...
        











































                                        Student
                                    

                                        Employer / T&P
                                    









                                    Login with Google
                                




OR






Email



Password



Forgot password?


Login



                                    New to Internshala? Register (Student / Company)
                                











                Your account is put on hold.


                Your account is put on hold.





In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Junior Data Scientist',
  'experience': '0 year(s)',
  'skills': ['Data Analytics', 'Data Science', 'Python', 'SQL'],
  'description': "As a junior data scientist, you will have the opportunity to work on cutting-edge projects and make a significant impact on the company's success. We are looking for a talented individual who is proficient in Python, Data Analytics, Data Science, and SQL."}]

In [11]:
type(json_res)

list

In [25]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [27]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [35]:
# 1. Get job entry from parsed JSON
job = json_res[0]
# 2. Query ChromaDB using job['skills']
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
print(links)


[[None, {'links': 'https://efds.fa.em5.oraclecloud.com/hcmUI/CandidateExperience/en/sites/CX_1/job/47287?utm_medium=jobboard&utm_source=indeed'}], [{'links': 'https://jobs.spectrum.com/job/-/-/4673/82984903264?utm_source=indeed&utm_medium=job-listings&utm_campaign=brand&utm_term=RD-TA'}, {'links': 'https://aoins.wd5.myworkdayjobs.com/en-US/AutoOwners/job/Lansing-MI/Data-Scientist_R_4694?source=AO_Indeed'}], [None, None], [None, {'links': 'https://careers-bluehawk.icims.com/jobs/2656/job?utm_source=indeed_integration&iis=Job%20Board&iisn=Indeed&indeed-apply-token=73a2d2b2a8d6d5c0a62696875eaebd669103652d3f0c2cd5445d3e66b1592b0f&mode=job&iis=Indeed&iisn=Indeed.com'}]]


In [37]:
job

{'role': 'Junior Data Scientist',
 'experience': '0 year(s)',
 'skills': ['Data Analytics', 'Data Science', 'Python', 'SQL'],
 'description': "As a junior data scientist, you will have the opportunity to work on cutting-edge projects and make a significant impact on the company's success. We are looking for a talented individual who is proficient in Python, Data Analytics, Data Science, and SQL."}

In [41]:
job = json_res[0]  #  get the first job dictionary
print(job['skills'])

['Data Analytics', 'Data Science', 'Python', 'SQL']


In [53]:
prompt_email = PromptTemplate.from_template(
        """
     ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Science Solutions for Your Organization

Dear Hiring Manager,

I came across your job posting for a Junior Data Scientist and was impressed by your company's commitment to leveraging cutting-edge projects to drive success. As a Business Development Executive at AtliQ, I'd like to introduce you to our AI & Software Consulting company, dedicated to facilitating seamless business process integration through automated tools.

Our team of experts has empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. We believe our capabilities align perfectly with your requirements, and we'd be delighted to discuss how we can support your organization.

Our portfolio showcases our expertise in data science and analytics, with successful projects such as:

* Developing a predictive analytics platform for a leading insurance company, resulting in a 25% reduction in claims processing time 